In [286]:
import os
import pandas as pd
import numpy as np
import pandas_profiling

In [287]:
DATA_DIR = 'data'
FILENAME = 'train.csv'

In [288]:
df = pd.read_csv(os.path.join(DATA_DIR, FILENAME))

C:\Users\Konstantin\.virtualenvs\bd_ml_itmo-1eel-7OV\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [289]:
df.head()

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
0,Пермь,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,...,5.762963,5.530612,1964.118519,1960.959184,Пермский край,32.0,S27289,2020-01-05,10,0
1,Шатура,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,...,2.894366,3.527778,1952.321678,1957.222222,Московская область,280.0,S17052,2020-01-05,10,0
2,Ярославль,NaN,COL_2,57.619140,39.850525,1,30,67,128,0,...,6.141414,7.222222,1968.150000,1973.370370,Ярославская область,297.4,S16913,2020-01-05,110,0
3,Новокузнецк,NaN,COL_3,53.897083,87.108604,0,0,5,21,0,...,8.581081,9.000000,1992.716216,2014.000000,Кемеровская область,190.0,S10148,2020-01-05,110,0
4,Москва,NaN,COL_4,55.802590,37.487110,1,23,64,153,0,...,7.263889,5.684211,1963.229167,1960.500000,Москва,60.2,S1338,2020-01-05,10,0


In [67]:
profile = pandas_profiling.ProfileReport(df, title='Profile report', minimal=True)

In [68]:
# profile.to_file("output.html")
profile.to_widgets()

Summarize dataset:   0%|          | 0/85 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [290]:
target_column = 'per_square_meter_price'
target = df[target_column]
df = df[[c for c in df.columns if c != target_column]]

In [291]:
list(df.columns)

['city',
 'floor',
 'id',
 'lat',
 'lng',
 'osm_amenity_points_in_0.001',
 'osm_amenity_points_in_0.005',
 'osm_amenity_points_in_0.0075',
 'osm_amenity_points_in_0.01',
 'osm_building_points_in_0.001',
 'osm_building_points_in_0.005',
 'osm_building_points_in_0.0075',
 'osm_building_points_in_0.01',
 'osm_catering_points_in_0.001',
 'osm_catering_points_in_0.005',
 'osm_catering_points_in_0.0075',
 'osm_catering_points_in_0.01',
 'osm_city_closest_dist',
 'osm_city_nearest_name',
 'osm_city_nearest_population',
 'osm_crossing_closest_dist',
 'osm_crossing_points_in_0.001',
 'osm_crossing_points_in_0.005',
 'osm_crossing_points_in_0.0075',
 'osm_crossing_points_in_0.01',
 'osm_culture_points_in_0.001',
 'osm_culture_points_in_0.005',
 'osm_culture_points_in_0.0075',
 'osm_culture_points_in_0.01',
 'osm_finance_points_in_0.001',
 'osm_finance_points_in_0.005',
 'osm_finance_points_in_0.0075',
 'osm_finance_points_in_0.01',
 'osm_healthcare_points_in_0.005',
 'osm_healthcare_points_in_0.

In [292]:
# Отбросить признаки с большим числом пропусков
print((df.isna().astype(int).sum(0) / df.shape[0]).sort_values()[-10:])

features_to_drop = ['floor', 'id']
df = df[[c for c in df.columns if c not in features_to_drop]]

price_type                        0.000000
osm_city_nearest_population       0.000197
street                            0.005740
reform_house_population_1000      0.052167
reform_mean_year_building_1000    0.058040
reform_mean_floor_count_1000      0.059716
reform_house_population_500       0.097337
reform_mean_year_building_500     0.105925
reform_mean_floor_count_500       0.107823
floor                             0.629886
dtype: float64


In [293]:
# Разделим данные на категории
categorical_features = ['price_type', 'realty_type', 'street', 'region', 'city']
numerical_featurs = [c for c in df.columns if c not in categorical_features]

In [294]:
df[[c for c in df[categorical_features] if df[c].dtype != 'O']] += 1
df1 = df.replace(np.nan, 0)

In [295]:
df1

,city,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,...,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
0,Пермь,57.998207,56.292797,4,19,35,52,0,0,0,...,5.762963,5.530612,1964.118519,1960.959184,Пермский край,32.000000,S27289,2020-01-05,11,1
1,Шатура,55.574284,39.543835,3,24,37,59,0,0,0,...,2.894366,3.527778,1952.321678,1957.222222,Московская область,280.000000,S17052,2020-01-05,11,1
2,Ярославль,57.619140,39.850525,1,30,67,128,0,0,1,...,6.141414,7.222222,1968.150000,1973.370370,Ярославская область,297.400000,S16913,2020-01-05,111,1
3,Новокузнецк,53.897083,87.108604,0,0,5,21,0,0,0,...,8.581081,9.000000,1992.716216,2014.000000,Кемеровская область,190.000000,S10148,2020-01-05,111,1
4,Москва,55.802590,37.487110,1,23,64,153,0,1,1,...,7.263889,5.684211,1963.229167,1960.500000,Москва,60.200000,S1338,2020-01-05,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279787,Томск,56.459183,84.979334,2,33,111,222,0,0,1,...,5.714286,5.882353,1972.260870,1973.460000,Томская область,358.320073,S11114,2020-08-23,11,2
279788,Санкт-Петербург,59.936954,30.356383,10,274,718,1340,0,2,5,...,4.719388,4.706667,1876.994898,1873.186667,Санкт-Петербург,119.637556,S28440,2020-08-23,111,2
279789,Калининград,54.729233,20.514968,0,12,34,84,0,2,2,...,3.950413,4.885714,1964.258333,1970.571429,Калининградская область,312.789725,S6671,2020-08-23,11,2
279790,Кемерово,55.360680,86.081460,5,57,100,134,0,0,0,...,4.691489,4.125000,1957.425532,1954.625000,Кемеровская область,89.201305,S17667,2020-08-23,111,2


In [230]:
# Отфильтруем данные по значениям целевой переменной
target_threshold = np.quantile(target.to_numpy(), 0.95)
filter_idx = np.where(target.to_numpy() < target_threshold)
target = target.iloc[filter_idx].reset_index(drop=True)
df = df.iloc[filter_idx].reset_index(drop=True)

In [231]:
# fig, ax = plt.subplots(1, figsize=(28,14))
# target[target < target_threshold].plot(ax=ax, ls='', marker='.', alpha=0.5)  # .hist(ax=ax)
# target.plot(ax=ax, ls='', marker='.', alpha=0.5)  # .hist(ax=ax, alpha=0.7)

In [258]:
facilities_in_raduis_columns = [c for c in df.columns if ('osm' in c and 'points' in c)]
category_names = np.unique([c.split('_in_')[0] for c in facilities_in_raduis_columns])
category_column_names = [[c for c in facilities_in_raduis_columns if category_name in c] for category_name in category_names]

In [269]:
tmp = []
for columns in category_column_names:
    columns = list(reversed(columns))
    facilities_num = df[columns]
    facilities_per_zone_num = facilities_num.iloc[:, 0: -1] - facilities_num.iloc[:, 1:]
    tmp.extend(facilities_per_zone_num)

# for i in range(0, len(facilities_in_raduis_columns), 4):
#     cols = facilities_in_raduis_columns[i: i + 4]
#     facilities = df[cols]
#     facilities_per_zone = facilities.iloc[:, 1:] - facilities.iloc[:, 0:-1]
# # df[facilities_in_raduis_columns] = tmp
        

# # groups = []
# # prev = facilities_in_raduis_columns[0]
# # for e in facilities_in_raduis_columns:
    

In [270]:
list(reversed(columns))

['osm_transport_stop_points_in_0.005',
 'osm_transport_stop_points_in_0.0075',
 'osm_transport_stop_points_in_0.01']

In [271]:
facilities_num.iloc[:, -1:] - facilities_num.iloc[:, 0:-1]

,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
265797,NaN,NaN,NaN
265798,NaN,NaN,NaN
265799,NaN,NaN,NaN
265800,NaN,NaN,NaN


In [191]:
for a, b in zip(facilities_in_raduis_columns, tmp):
    print(a,b)

,price_type,realty_type,street,region,city
0,0,10,S27289,Пермский край,Пермь
1,0,10,S17052,Московская область,Шатура
2,0,110,S16913,Ярославская область,Ярославль
3,0,110,S10148,Кемеровская область,Новокузнецк
4,0,10,S1338,Москва,Москва
...,...,...,...,...,...
279786,1,110,S22612,Калужская область,Калуга
279787,1,10,S11114,Томская область,Томск
279789,1,10,S6671,Калининградская область,Калининград
279790,1,110,S17667,Кемеровская область,Кемерово


In [196]:
cond1 = df.floor.notna()
cond2 = df.floor == 1
cond3 = df.floor != 1

df.floor[cond1 & cond2]

0         True
1         True
2         True
3         True
4         True
          ... 
279786    True
279787    True
279789    True
279790    True
279791    True
Name: price_type, Length: 265802, dtype: bool